#### **This notebook contains the functions used to clean the spectra.**

Import Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold, cross_validate, GroupKFold
import seaborn as sns
from Spectra_Preparation_Functions import *
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import peak_widths, peak_prominences


Read the spectral data

In [2]:
df = pd.read_csv("../../data/groupkfold_parameters.csv")


In [3]:
df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance
0,201210-1-00,293,400.22778,201210-1,Normal,41.863303
1,201210-1-00,294,400.91116,201210-1,Normal,41.803843
2,201210-1-00,295,401.59454,201210-1,Normal,41.741884
3,201210-1-00,296,402.27789,201210-1,Normal,41.677722
4,201210-1-00,297,402.96127,201210-1,Normal,41.611654
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,12.378163
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,13.269937
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,14.199285
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,15.166531


In [4]:
def evaluate_model(df, model):

    # Set the Surfaces as groups
    groups = df['SurID']
    X = df.drop(['Status', 'SurID'], axis=1)
    y = df['Status']
    
    # Using GroupKFold for classification tasks
    cv = GroupKFold(n_splits=10)

    # Cross Validate
    scores = cross_validate(model, X, y, groups=groups, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], n_jobs=-1)

    # Displaying the results
    print(f"{model.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores['test_accuracy']):.4f} +/- {np.std(scores['test_accuracy']):.4f}")
    print(f"{model.__class__.__name__} Cross-Validation Precision: {np.mean(scores['test_precision_macro']):.4f} +/- {np.std(scores['test_precision_macro']):.4f}")
    print(f"{model.__class__.__name__} Cross-Validation Recall: {np.mean(scores['test_recall_macro']):.4f} +/- {np.std(scores['test_recall_macro']):.4f}")
    print(f"{model.__class__.__name__} Cross-Validation F1-Score: {np.mean(scores['test_f1_macro']):.4f} +/- {np.std(scores['test_f1_macro']):.4f}")

In [5]:
def prepare_wavelength_df(df, absorbance_col, status_col='Status'):

    # Pivot the DataFrame to get wavelengths as columns and absorbance values
    wavelength_df = df.pivot(index='SpecID', columns='WaveNumber', values=absorbance_col).reset_index()
    wavelength_df.columns.name = None

    # Merge with the statuses based on SpecID
    # Include the SurID to perform GroupKFold CV
    statuses_and_surface = df[['SpecID', 'SurID', status_col]].drop_duplicates()
    wavelength_df = pd.merge(wavelength_df, statuses_and_surface, on='SpecID')

    # Set SpecID as the index
    wavelength_df = wavelength_df.set_index('SpecID')

    return wavelength_df

In [6]:
wavelength_df = prepare_wavelength_df(df, 'Absorbance')

In [7]:
et = ExtraTreesClassifier(random_state=1234)
rf = RandomForestClassifier(random_state=1234)
svc = SVC(random_state=1234)

##### **Full Spectra**

In [8]:
evaluate_model(wavelength_df, et)

ExtraTreesClassifier Cross-Validation Accuracy: 0.6119 +/- 0.1114
ExtraTreesClassifier Cross-Validation Precision: 0.6103 +/- 0.1103
ExtraTreesClassifier Cross-Validation Recall: 0.6215 +/- 0.1349
ExtraTreesClassifier Cross-Validation F1-Score: 0.5949 +/- 0.1254


/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
evaluate_model(wavelength_df, rf)

/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForestClassifier Cross-Validation Accuracy: 0.6122 +/- 0.0993
RandomForestClassifier Cross-Validation Precision: 0.6152 +/- 0.0926
RandomForestClassifier Cross-Validation Recall: 0.6216 +/- 0.1245
RandomForestClassifier Cross-Validation F1-Score: 0.5948 +/- 0.1121


In [10]:
evaluate_model(wavelength_df, svc)

/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVC Cross-Validation Accuracy: 0.4272 +/- 0.1406
SVC Cross-Validation Precision: 0.4800 +/- 0.1651
SVC Cross-Validation Recall: 0.4137 +/- 0.1754
SVC Cross-Validation F1-Score: 0.3656 +/- 0.1404


##### **Peak Stats**

In [15]:
peaks = []
widths = []
prominences = []
statuses = df[['SpecID', 'Status', 'SurID']].drop_duplicates()

# Find the index and width of each peak
for _, group in df.groupby('SpecID'):

    #peak_index, _ = find_peaks(x=group['Absorbance'], distance=152, prominence=42, width=6)
    peak_index, _ = find_peaks(x=group['Absorbance'])


    # Calculate the widths of each peak
    widths += list(peak_widths(group['Absorbance'], peaks=peak_index, rel_height=0.5)[0])

    # Calculate prominence of each peak
    prominences += list(peak_prominences(group['Absorbance'], peaks=peak_index)[0])

    # Find the index of the peak within the full dataframe
    peaks += list(group.iloc[peak_index].index.values)

peaks_df = df.iloc[peaks]
peaks_df['PeakWidths'] = widths
peaks_df['PeakProminences'] = prominences
peaks_df

# Create a new DataFrame for the summary statistics
peak_stats = peaks_df.groupby('SpecID').agg({
    'Absorbance': ['mean', 'std', 'count', 'max', 'min'],
    'PeakWidths': ['mean', 'std', 'max', 'min'],
    'PeakProminences': ['mean', 'std', 'max', 'min']
}).reset_index()

# Flatten the multi-level columns and customize the names
peak_stats.columns = ['SpecID',
                      'PeakAbsorbance_mean', 'PeakAbsorbance_std', 'PeakAbsorbance_count', 'PeakAbsorbance_max', 'PeakAbsorbance_min',
                      'PeakWidths_mean', 'PeakWidths_std', 'PeakWidths_max', 'PeakWidths_min',
                      'PeakProminences_mean', 'PeakProminences_std', 'PeakProminences_max', 'PeakProminences_min']

peak_stats.shape
peak_stats = pd.merge(statuses, peak_stats, on='SpecID', how='left')
peak_stats = peak_stats.set_index('SpecID')
peak_stats = peak_stats.fillna(0)

/tmp/ipykernel_38412/1114612660.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peaks_df['PeakWidths'] = widths
/tmp/ipykernel_38412/1114612660.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peaks_df['PeakProminences'] = prominences


In [16]:
peak_stats

,Status,SurID,PeakAbsorbance_mean,PeakAbsorbance_std,PeakAbsorbance_count,PeakAbsorbance_max,PeakAbsorbance_min,PeakWidths_mean,PeakWidths_std,PeakWidths_max,PeakWidths_min,PeakProminences_mean,PeakProminences_std,PeakProminences_max,PeakProminences_min
SpecID,,,,,,,,,,,,,,,
201210-1-00,Normal,201210-1,42.724182,31.686298,205,173.397843,-7.558160,5.754848,18.698699,194.114304,0.502925,4.502105,19.616381,169.566634,0.004136
201210-1-01,Normal,201210-1,45.522946,42.483632,183,188.679018,-10.658033,6.588338,19.723760,151.983867,0.503475,6.408729,26.918589,187.781984,0.003636
201210-1-02,Normal,201210-1,86.632065,130.929315,141,1336.381183,-25.851462,6.232133,12.990837,67.855151,0.503902,20.954275,126.097991,1341.050057,0.001224
201210-1-03,Normal,201210-1,122.589284,191.431125,114,987.344326,-51.984432,7.055155,15.043040,71.923318,0.501391,37.095520,160.515805,1000.595932,0.003340
201210-1-04,Normal,201210-1,67.658148,57.916299,189,306.170893,-17.099481,6.257609,20.057264,183.353131,0.501293,6.715678,30.324634,317.970476,0.003077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210526-3-05,Hyperglycemia,210526-3,59.340378,63.513923,154,453.251197,-2.236862,7.349668,16.882553,119.909577,0.502152,11.593617,46.123829,457.165115,0.001815
210526-3-06,Hyperglycemia,210526-3,56.953202,56.688423,190,485.468080,-3.875471,6.270059,15.710435,112.777156,0.511045,9.078047,42.079487,486.563979,0.013106
210526-3-07,Hyperglycemia,210526-3,51.472942,48.092297,148,349.507366,1.776651,7.720291,18.521906,131.018560,0.502461,9.690046,35.546181,349.983911,0.009252


In [17]:
evaluate_model(peak_stats, et)

ExtraTreesClassifier Cross-Validation Accuracy: 0.4056 +/- 0.0722
ExtraTreesClassifier Cross-Validation Precision: 0.3969 +/- 0.0666
ExtraTreesClassifier Cross-Validation Recall: 0.3959 +/- 0.0912
ExtraTreesClassifier Cross-Validation F1-Score: 0.3786 +/- 0.0780


/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
evaluate_model(peak_stats, rf)

RandomForestClassifier Cross-Validation Accuracy: 0.4168 +/- 0.0751
RandomForestClassifier Cross-Validation Precision: 0.4077 +/- 0.0711
RandomForestClassifier Cross-Validation Recall: 0.4052 +/- 0.0937
RandomForestClassifier Cross-Validation F1-Score: 0.3892 +/- 0.0819


/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
evaluate_model(peak_stats, svc)

SVC Cross-Validation Accuracy: 0.3712 +/- 0.1054
SVC Cross-Validation Precision: 0.3259 +/- 0.1458
SVC Cross-Validation Recall: 0.3549 +/- 0.1399
SVC Cross-Validation F1-Score: 0.2903 +/- 0.0940


/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/eddie/.local/lib/python3.10

##### **Peak Bins**

In [20]:
# Define a function to calculate the bin for a given wavenumber with a specified bin size
def calculate_bin_interval(wavenumber, bin_size):
    bin_start = int((wavenumber - 200) / bin_size) * bin_size + 200
    bin_end = bin_start + bin_size
    return f"{bin_start}-{bin_end}"
# Set the bin size
bin_size = 25

# Add a "Bin" column to the DataFrame
peaks_df['Bin'] = peaks_df['WaveNumber'].apply(lambda x: calculate_bin_interval(x, bin_size))
peaks_df.head()

# Set the bins as columns with the peak absrobances, widths and prominences as the values.
# If no peaks appear in a bin the value is set to 0.
# If multiple peaks appear their properties are aggregated.

# Pivot table with 'Absorbance', 'PeakWidths', and 'PeakProminences' as values
peak_bins = peaks_df.pivot_table(index='SpecID', columns='Bin', values=['Absorbance', 'PeakWidths', 'PeakProminences'], aggfunc='mean')
peak_bins.columns = [f"{col[0]}_{col[1]}" for col in peak_bins.columns]  # Combine column names
peak_bins.reset_index(inplace=True)

# Merge with 'Status' information
statuses = peaks_df[['SpecID', 'Status', 'SurID']].drop_duplicates()
peak_bins = pd.merge(peak_bins, statuses, on='SpecID')

# Set 'SpecID' as the index
peak_bins.set_index('SpecID', inplace=True)

# Fill NaN values with 0
peak_bins.fillna(0, inplace=True)
peak_bins.head()

/tmp/ipykernel_38412/2330773350.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peaks_df['Bin'] = peaks_df['WaveNumber'].apply(lambda x: calculate_bin_interval(x, bin_size))


,Absorbance_1000-1025,Absorbance_1025-1050,Absorbance_1050-1075,Absorbance_1075-1100,Absorbance_1100-1125,Absorbance_1125-1150,Absorbance_1150-1175,Absorbance_1175-1200,Absorbance_1200-1225,Absorbance_1225-1250,...,PeakWidths_800-825,PeakWidths_825-850,PeakWidths_850-875,PeakWidths_875-900,PeakWidths_900-925,PeakWidths_925-950,PeakWidths_950-975,PeakWidths_975-1000,Status,SurID
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,89.002395,67.967842,66.465735,54.529033,20.073329,34.716640,21.469727,4.954262,6.181741,45.558273,...,6.069429,0.000000,8.584305,31.642064,0.000000,2.131577,0.670059,0.000000,Normal,201210-1
201210-1-01,99.991163,50.110842,65.076935,47.419326,18.229328,36.275023,28.967117,21.372376,-2.406472,8.481125,...,1.155223,0.000000,3.797847,0.826808,143.050278,17.800870,0.556733,0.000000,Normal,201210-1
201210-1-02,176.303245,197.680777,81.129202,73.209240,28.286036,81.935506,15.763898,7.150480,17.239728,25.321297,...,3.840262,0.000000,1.694357,0.000000,35.976770,0.000000,4.762265,1.079848,Normal,201210-1
201210-1-03,403.941153,0.000000,231.782303,0.000000,138.570161,0.000000,637.018793,0.000000,41.350087,82.255296,...,0.524564,1.296502,0.000000,0.000000,70.641586,0.000000,29.168888,1.046836,Normal,201210-1
201210-1-04,98.087538,47.702445,46.462610,16.671599,15.590154,37.450891,52.928517,76.141247,27.652156,21.135276,...,0.504633,0.935608,16.933157,1.907858,2.417041,1.261868,0.000000,1.286921,Normal,201210-1


In [21]:
evaluate_model(peak_bins, et)

ExtraTreesClassifier Cross-Validation Accuracy: 0.5498 +/- 0.1029
ExtraTreesClassifier Cross-Validation Precision: 0.5584 +/- 0.0954
ExtraTreesClassifier Cross-Validation Recall: 0.5609 +/- 0.1288
ExtraTreesClassifier Cross-Validation F1-Score: 0.5250 +/- 0.1136


/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
evaluate_model(peak_bins, rf)

RandomForestClassifier Cross-Validation Accuracy: 0.5519 +/- 0.1081
RandomForestClassifier Cross-Validation Precision: 0.5577 +/- 0.0993
RandomForestClassifier Cross-Validation Recall: 0.5670 +/- 0.1298
RandomForestClassifier Cross-Validation F1-Score: 0.5314 +/- 0.1171


/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [23]:
evaluate_model(peak_bins, svc)

/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVC Cross-Validation Accuracy: 0.4475 +/- 0.1452
SVC Cross-Validation Precision: 0.5055 +/- 0.1218
SVC Cross-Validation Recall: 0.4429 +/- 0.1789
SVC Cross-Validation F1-Score: 0.3871 +/- 0.1390
